In [1]:
import pandas as pd
import sys
import numpy as np
sys.path.append('../')


import nlp
import wrangle

import nltk

from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_columns', None)
datadf, dictionarydf = wrangle.wrangle_data(path_prefix='../')

In [3]:
datadf

,job_title,job_taxo,job_id,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,num_employees,num_researchers,primary_industry,types_res_used,future_res,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,research_educ,research_educ_cat,research_educ_desc,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,how_pick_events,best_event,events_attend_recent,ideal_conference_size,ideal_conference_size_cat_orig,ideal_conference_size_cat,ideal_structure,ideal_structure_cat,ideal_multi_track,ideal_single_track,ideal_unconference,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables,other_conference_types,ideal_topics,ideal_attendees,recommendations,persona_id
resp_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
284,Academic,4,1.0,3,2,0,3,3,3,3,2,1,4,0.0,2.0,maritime,both qualitative and quantitative research,both qualitative and quantitative research,5,5,5,5,5,5,5,5,5,5,"sort of, psychology background.",1.0,NaN,0,1,1,2,3,1,3,3,2,3,3,4,4,3,3,4,2,3,1,3,1,3,4,NaN,NaN,"CHI, INTERACT",depending on the professional values,7.0,7.0,"any, depending on the values",5.0,0,0,0,4,2,2,2,3,2,2,2,NaN,research-practice connections,NaN,research-practice connections,2
288,Academic,4,1.0,1,2,1,3,3,3,3,3,0,3,4.0,3.0,Higher education,Multiple,Multiple,5,5,5,5,5,5,5,5,2,5,"Yes, I was taught how to conduct research",1.0,Research methods and statistics training in bo...,2,3,2,2,2,2,3,3,2,3,2,2,2,3,3,4,3,4,2,3,3,2,4,"Content, cost, timing (can I get away at this ...","Dagstuhl workshop - small, intense, intimate g...",Hawaii international conference on Education,I would attend any of these,6.0,6.0,Multi-track: Every attendee chooses their own ...,2.0,1,0,0,3,3,3,3,3,1,1,2,NaN,NaN,NaN,Don’t create schedule conflicts with other rel...,2
294,Academic,4,1.0,3,3,1,3,1,3,3,3,1,3,4.0,3.0,University,"User research / Design research\n\nInterviews,...",Online FG,4,4,3,3,4,4,4,4,3,3,"Yes, I was taught how to conduct research",1.0,BA Sociology\nMA LIS\nPhD. LIS\n\nonline cours...,2,3,2,2,2,3,4,3,3,3,3,3,3,3,3,4,4,3,4,3,3,4,3,- time for travel (having two small children]\...,Courses at Copenhagen University (deep knowle...,"Research Tuesdays (Czech), UX/Design Conferen...",It depends - I really enjoyed small conference...,7.0,7.0,It depends,5.0,0,0,0,2,2,2,2,2,2,2,2,NaN,NaN,NaN,Looking forward! \n,2
281,Academic,4,1.0,3,4,0,2,3,2,0,1,0,0,2.0,1.0,Ed Tech,"surveys, interviews, focus groups, secondary d...","surveys, interviews, focus groups, secondary d...",1,1,0,1,1,1,0,1,0,0,"Yes, I was taught how to conduct research",1.0,"I have an Applied Anthropology, MA. I designed...",1,2,0,1,2,3,4,3,2,4,2,3,3,2,3,3,3,1,1,1,2,3,4,My professional development budget allows me t...,A workshop at the Society for Applied Anthropo...,A local education implementation conference.,Large conference: 500+ attendees,4.0,4.0,Multi-track: Every attendee chooses their own ...,2.0,1,0,0,4,2,3,3,2,2,3,2,NaN,"Research design, innovative methods, communica...",NaN,NaN,2
280,Academic,4,1.0,3,3,0,1,1,3,2,1,0,0,NaN,NaN,education,NaN,NaN,4,4,0,2,2,3,2,3,0,0,"No, I was not taught how to conduct research",0.0,NaN,1,1,1,1,2,3,3,3,1,2,1,2,1,3,1,4,2,3,1,2,2,4,4,NaN,NaN,NaN,Large conference: 500+ attendees,4.0,4.0,Multi-track: Every attendee chooses their own ...,2.0,1,0,0,4,2,3,3,2,1,2,2,You should consider the combination of confere...,NaN,

In [4]:
datadf.describe()

,job_conduct_res,job_analyze_res,job_buy_res_report,job_manage_res_proj,job_observe_res,job_plan_res,job_teach_res,job_advocate_res,job_hire_res_vendor,job_lead_res_team,exp_conduct_res,exp_analyze_res,exp_buy_res_report,exp_manage_res_proj,exp_observe_res,exp_plan_res,exp_teach_res,exp_advocate_res,exp_hire_res_vendor,exp_lead_res_team,learning_talks,learning_read,learning_meetup,learning_workshop,learning_conference,likely_watch_video,likely_internet,likely_book,likely_online_group,likely_colleague,likely_meetup,likely_conference,likely_workshop,factor_speaker,factor_diverse_speak,factor_topics,factor_format_sessions,factor_size,factor_network,factor_variety_attend,factor_code,factor_location,factor_ability_to_pay,ideal_multi_track,ideal_single_track,ideal_unconference,session_keynote,session_long_talk,session_short_talk,session_workshop,session_network,session_social_event,session_qa,sesson_topic_tables
count,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000,726.000000
mean,2.856749,2.913223,0.402204,2.600551,1.982094,2.931129,1.845730,2.924242,0.776860,1.860882,3.283747,3.323691,0.960055,2.633609,3.084022,3.139118,1.862259,2.906336,1.326446,1.632231,2.074380,2.659780,1.801653,1.451791,1.663912,2.982094,3.500000,3.263085,2.079890,3.053719,2.202479,2.250689,2.360882,2.236915,2.871901,3.652893,2.776860,2.209366,1.998623,2.028926,1.903581,2.944904,3.336088,0.625344,0.334711,0.061983,2.706612,2.754821,2.341598,2.988981,2.173554,1.480716,2.121212,2.431129
std,1.025941,0.939210,0.720686,1.166285,1.134273,0.950177,1.236637,1.056235,0.914143,1.486931,1.571974,1.553165,1.608530,1.702955,1.694565,1.589165,1.684913,1.711475,1.654099,1.674476,0.791318,0.571707,0.836609,0.784165,0.761119,0.960436,0.711967,0.834391,1.213355,0.983242,1.150254,1.143723,1.101378,1.157435,1.071954,0.590183,0.926135,1.168658,1.107341,1.158115,1.263411,1.019666,0.971309,0.484368,0.472215,0.241292,1.124344,0.977229,1.106273,1.109154,1.219458,1.239149,1.179908,1.211472
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,0.000000,2.000000,1.000000,3.000000,1.000000,3.000000,0.000000,0.000000,2.000000,2.000000,0.000000,1.000000,2.000000,2.000000,0.000000,1.000000,0.000000,0.000000,2.000000,2.000000,1.000000,1.000000,1.000000,3.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,1.000000,2.000000,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.000000,1.000000,2.000000
50%,3.000000,3.000000,0.000000,3.000000,2.000000,3.000000,2.000000,3.000000,1.000000,2.000000,4.000000,4.000000,0.000000,2.000000,3.000000,3.000000,1.000000,3.000000,1.000000,1.000000,2.000000,3.000000,2.000000,1.000000,2.000000,3.000000,4.000000,3.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,4.000000,1.000000,0.000000,0.000000,3.000000,3.000000,3.000000,3.000000,2.000000,1.000000,2.000000,3.000000
75%,4.000000,3.000000,1.00000

In [5]:
datadf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 726 entries, 284 to 2
Data columns (total 78 columns):
job_title                         726 non-null category
job_taxo                          726 non-null category
job_id                            723 non-null category
job_conduct_res                   726 non-null int64
job_analyze_res                   726 non-null int64
job_buy_res_report                726 non-null int64
job_manage_res_proj               726 non-null int64
job_observe_res                   726 non-null int64
job_plan_res                      726 non-null int64
job_teach_res                     726 non-null int64
job_advocate_res                  726 non-null int64
job_hire_res_vendor               726 non-null int64
job_lead_res_team                 726 non-null int64
num_employees                     723 non-null category
num_researchers                   722 non-null category
primary_industry                  726 non-null category
types_res_used               

## 5. What is your company or organization's primary industry?

In [52]:
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.basic_clean)
datadf.primary_industry = datadf.primary_industry.dropna().apply(nlp.lemmatize)

In [53]:
primary_industry_matrix, primary_industry_vector = nlp.create_wordcount_matrix(datadf.primary_industry.dropna(), ngram=(1,3), max_df=.5)
primary_industry_matrix, primary_industry_vector

(<726x180 sparse matrix of type '<class 'numpy.int64'>'
 	with 1239 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [54]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [55]:
lda22.fit(primary_industry_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [56]:
pyLDAvis.sklearn.prepare(lda22, primary_industry_matrix, primary_industry_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.210977 -0.162990       1        1  38.649390
2     -0.249731 -0.097304       2        1  33.882675
0      0.038754  0.260294       3        1  27.467936, topic_info=    Category       Freq                 Term      Total  loglift  logprob
157  Default  41.000000                 tech  41.000000  30.0000  30.0000
151  Default  40.000000             software  40.000000  29.0000  29.0000
75   Default  46.000000           healthcare  46.000000  28.0000  28.0000
67   Default  38.000000              fintech  38.000000  27.0000  27.0000
26   Default  35.000000           consulting  35.000000  26.0000  26.0000
159  Default  38.000000           technology  38.000000  25.0000  25.0000
51   Default  35.000000            education  35.000000  24.0000  24.0000
71   Default  28.000000           government  28.000000  23.0000  23.0000
86   Default  23.000000             industry  23.000000  22.0000  22.0000
65   Default  23.000000            financial  23.000000  21.0000  21.0000
142  Default  23.000000               retail  23.000000  20.0000  20.0000
143  Default  15.000000                 saas  15.000000  19.0000  19.0000
136  Default  21.000000             research  21.000000  18.0000  18.0000
23   Default  14.000000          consultancy  14.000000  17.0000  17.0000
92   Default  16.000000            insurance  16.000000  16.0000  16.0000
148  Default  37.000000              service  37.000000  15.0000  15.0000
66   Default  18.000000    financial service  18.000000  14.0000  14.0000
111  Default  15.000000               medium  15.000000  13.0000  13.0000
43   Default  14.000000              digital  14.000000  12.0000  12.0000
47   Default  15.000000            ecommerce  15.000000  11.0000  11.0000
118  Default  10.000000            nonprofit  10.000000  10.0000  10.0000
128  Default  10.000000               public  10.000000   9.0000   9.0000
55   Default  10.000000           enterprise  10.000000   8.0000   8.0000
146  Default   9.000000               sector   9.000000   7.0000   7.0000
63   Default  13.000000              finance  13.000000   6.0000   6.0000
39   Default  29.000000               design  29.000000   5.0000   5.0000
2    Default  15.000000               agency  15.000000   4.0000   4.0000
77   Default   9.000000               higher   9.000000   3.0000   3.0000
7    Default   9.000000              banking   9.000000   2.0000   2.0000
133  Default   9.000000                 real   9.000000   1.0000   1.0000
..       ...        ...                  ...        ...      ...      ...
55    Topic3   9.680832           enterprise  10.263468   1.2337  -3.5662
146   Topic3   8.727224               sector   9.322885   1.2261  -3.6699
107   Topic3   6.864622            marketing   7.448263   1.2106  -3.9100
74    Topic3   6.862561               health   7.448688   1.2102  -3.9103
56    Topic3   5.939852  enterprise software   6.511743   1.2002  -4.0547
42    Topic3   5.912632          development   6.509340   1.1960  -4.0593
50    Topic3   5.000976               edtech   5.573465   1.1838  -4.2267
160   Topic3   5.000658              telecom   5.573438   1.1837  -4.2268
129   Topic3   4.994836        public sector   5.572639   1.1827  -4.2279
154   Topic3   4.064626     software service   4.635434   1.1607  -4.4340
81    Topic3   4.064116                   hr   4.635395   1.1606  -4.4341
115   Topic3   4.062258             multiple   4.635292   1.1602  -4.4346
82    Topic3   3.126472              hr tech   3.697258   1.1245  -4.6964
97    Topic3   3.125312                legal   3.697160   1.1241  -4.6968
73    Topic3   3.124703             hardware   3.697110   1.1239  -4.6970
4     Topic3   3.120095           automotive   3.696885   1.1225  -4.6985
69    Topic3   3.111671            freelance   3.696387   1.1200  -4.7012
36    Topic3   2.188474                 data   2.759090   1.0

In [57]:
lda_W22 = lda22.transform(primary_industry_matrix)

In [59]:
top_doc_column = datadf.primary_industry.dropna()

In [63]:
word_count_matrix2, count_vect2 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [64]:
LDA2 = LatentDirichletAllocation(n_components=5, random_state=42)
LDA2.fit(word_count_matrix2)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [65]:
lda_H = LDA2.transform(word_count_matrix2)

In [66]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column, 3)

Top 3 Documents for Topic 0: 

Document 1
real estate tech

Document 2
real estate tech

Document 3
sotware development across domain such a data science deep learning health tech interaction etc

Top 3 Documents for Topic 1: 

Document 1
design research service and innovation planning aka design planning design ledstrategy

Document 2
market research agency 95 client are in financial service

Document 3
ux research we're a ux research agency a vendor used by fortune 500 company

Top 3 Documents for Topic 2: 

Document 1
commercial real estate

Document 2
commercial real estate

Document 3
architecture engineering construction aec industry manufacturing design medium entertainment industry

Top 3 Documents for Topic 3: 

Document 1
banking retail edu logistic government entertainment tourism aviation

Document 2
financial sector public sector resource technology retail healthcare

Document 3
aerospaceautomativebankingcentral government chemical consumer package goodsdefenceeducation ma

## 6. What types of research do you currently use to make decisions?

In [ ]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [ ]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.5)
ideal_att_matrix, ideal_att_vector

In [ ]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda22.fit(ideal_att_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

In [ ]:
lda_W22 = lda22.transform(primary_industry_matrix)

In [ ]:
top_doc_column = datadf.primary_industry.dropna()

In [ ]:
word_count_matrix2, count_vect2 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA2 = LatentDirichletAllocation(n_components=5, random_state=42)
LDA2.fit(word_count_matrix2)

In [ ]:
lda_H = LDA2.transform(word_count_matrix2)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column, 3)

## 7. What types of research are you considering in the future?

In [ ]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [ ]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.5)
ideal_att_matrix, ideal_att_vector

In [ ]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda22.fit(ideal_att_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

In [ ]:
lda_W22 = lda22.transform(primary_industry_matrix)

In [ ]:
top_doc_column = datadf.primary_industry.dropna()

In [ ]:
word_count_matrix2, count_vect2 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA2 = LatentDirichletAllocation(n_components=5, random_state=42)
LDA2.fit(word_count_matrix2)

In [ ]:
lda_H = LDA2.transform(word_count_matrix2)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column, 3)

## 10. Describe your educational background with research

In [ ]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [ ]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.5)
ideal_att_matrix, ideal_att_vector

In [ ]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda22.fit(ideal_att_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

In [ ]:
lda_W22 = lda22.transform(primary_industry_matrix)

In [ ]:
top_doc_column = datadf.primary_industry.dropna()

In [ ]:
word_count_matrix2, count_vect2 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA2 = LatentDirichletAllocation(n_components=5, random_state=42)
LDA2.fit(word_count_matrix2)

In [ ]:
lda_H = LDA2.transform(word_count_matrix2)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column, 3)

## 14. How do you decide which events to attend?

In [ ]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [ ]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.5)
ideal_att_matrix, ideal_att_vector

In [ ]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda22.fit(ideal_att_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

In [ ]:
lda_W22 = lda22.transform(primary_industry_matrix)

In [ ]:
top_doc_column = datadf.primary_industry.dropna()

In [ ]:
word_count_matrix2, count_vect2 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA2 = LatentDirichletAllocation(n_components=5, random_state=42)
LDA2.fit(word_count_matrix2)

In [ ]:
lda_H = LDA2.transform(word_count_matrix2)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column, 3)

## 15. What was the best professional learning experience you've ever had?  What made it great?

In [ ]:
datadf.best_event = datadf.best_event.dropna().apply(nlp.basic_clean)
datadf.best_event = datadf.best_event.dropna().apply(nlp.lemmatize)

In [ ]:
best_event_matrix, best_event_vector = nlp.create_wordcount_matrix(datadf.best_event.dropna(), ngram=(1,3), max_df=.5)
best_event_matrix, best_event_vector

In [ ]:
lda15 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda15.fit(best_event_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda15, best_event_matrix, best_event_vector)

In [ ]:
lda_W15 = lda15.transform(best_event_matrix)

In [ ]:
top_doc_column = datadf.primary_industry.dropna()

In [ ]:
word_count_matrix2, count_vect2 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA2 = LatentDirichletAllocation(n_components=5, random_state=42)
LDA2.fit(word_count_matrix2)

In [ ]:
lda_H = LDA2.transform(word_count_matrix2)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column, 3)

## 16. What if any events have you attended on the subject of research in the past few years?

In [ ]:
datadf.best_event = datadf.best_event.dropna().apply(nlp.basic_clean)
datadf.best_event = datadf.best_event.dropna().apply(nlp.lemmatize)

In [ ]:
lda15 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda15.fit(best_event_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda15, best_event_matrix, best_event_vector)

In [ ]:
lda_W15 = lda15.transform(best_event_matrix)

In [ ]:
top_doc_column = datadf.primary_industry.dropna()

In [ ]:
word_count_matrix2, count_vect2 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA2 = LatentDirichletAllocation(n_components=5, random_state=42)
LDA2.fit(word_count_matrix2)

In [ ]:
lda_H = LDA2.transform(word_count_matrix2)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column, 3)

## 20. Did we miss any other types of conference sessions that you'd like to mention?

In [ ]:
datadf.other_conference_types = datadf.other_conference_types.dropna().apply(nlp.basic_clean)
datadf.other_conference_types = datadf.other_conference_types.dropna().apply(nlp.lemmatize)

In [ ]:
other_conference_types_matrix, other_conference_types_vector = nlp.create_wordcount_matrix(datadf.other_conference_types.dropna(), ngram=(1,3), max_df=.5)
other_conference_types_matrix, other_conference_types_vector

In [ ]:
lda20 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [ ]:
lda20.fit(other_conference_types_matrix)

In [ ]:
pyLDAvis.sklearn.prepare(lda20, other_conference_types_matrix, other_conference_types_vector)

In [ ]:
lda_W20 = lda20.transform(other_conference_types_matrix)

In [ ]:
top_doc_column = datadf.primary_industry.dropna()

In [ ]:
word_count_matrix2, count_vect2 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

In [ ]:
LDA2 = LatentDirichletAllocation(n_components=5, random_state=42)
LDA2.fit(word_count_matrix2)

In [ ]:
lda_H = LDA2.transform(word_count_matrix2)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column, 3)

## 22. If attending a conference about research, who might you be excited to see there?

In [37]:
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.basic_clean)
datadf.ideal_attendees = datadf.ideal_attendees.dropna().apply(nlp.lemmatize)

In [38]:
ideal_att_matrix, ideal_att_vector = nlp.create_wordcount_matrix(datadf.ideal_attendees.dropna(), ngram=(1,3), max_df=.5)
ideal_att_matrix, ideal_att_vector

(<471x925 sparse matrix of type '<class 'numpy.int64'>'
 	with 4580 stored elements in Compressed Sparse Row format>,
 CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                 dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                 lowercase=True, max_df=0.5, max_features=None, min_df=2,
                 ngram_range=(1, 3), preprocessor=None, stop_words='english',
                 strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                 tokenizer=None, vocabulary=None))

In [39]:
lda22 = LatentDirichletAllocation(n_components= 3, random_state = 42)

In [40]:
lda22.fit(ideal_att_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [41]:
pyLDAvis.sklearn.prepare(lda22, ideal_att_matrix, ideal_att_vector)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.122573  0.065619       1        1  46.076154
2      0.007800 -0.120126       2        1  28.072781
1     -0.130373  0.054507       3        1  25.851065, topic_info=    Category        Freq                  Term       Total  loglift  logprob
668  Default  179.000000              research  179.000000  30.0000  30.0000
557  Default  188.000000                people  188.000000  29.0000  29.0000
514  Default   18.000000                   new   18.000000  28.0000  28.0000
708  Default   15.000000            sam ladner   15.000000  27.0000  27.0000
406  Default   15.000000                ladner   15.000000  26.0000  26.0000
707  Default   15.000000                   sam   15.000000  25.0000  25.0000
775  Default   13.000000                 spool   13.000000  24.0000  24.0000
922  Default   16.000000                 young   16.000000  23.0000  23.0000
380  Default   12.000000           jared spool   12.000000  22.0000  22.0000
195  Default   26.000000                 doing   26.000000  21.0000  21.0000
348  Default   12.000000                  indi   12.000000  20.0000  20.0000
349  Default   12.000000            indi young   12.000000  19.0000  19.0000
836  Default   11.000000                 topic   11.000000  18.0000  18.0000
379  Default   13.000000                 jared   13.000000  17.0000  17.0000
786  Default   22.000000                 steve   22.000000  16.0000  16.0000
4    Default   24.000000              academic   24.000000  15.0000  15.0000
353  Default   41.000000              industry   41.000000  14.0000  14.0000
602  Default   18.000000              portigal   18.000000  13.0000  13.0000
788  Default   17.000000        steve portigal   17.000000  12.0000  12.0000
532  Default    8.000000                norman    8.000000  11.0000  11.0000
308  Default   15.000000                  hall   15.000000  10.0000  10.0000
303  Default   12.000000                 group   12.000000   9.0000   9.0000
853  Default   12.000000                  type   12.000000   8.0000   8.0000
190  Default   13.000000             diversity   13.000000   7.0000   7.0000
811  Default   20.000000                  team   20.000000   6.0000   6.0000
422  Default    8.000000          leader field    8.000000   5.0000   5.0000
226  Default   14.000000                 erika   14.000000   4.0000   4.0000
158  Default    9.000000                   day    9.000000   3.0000   3.0000
547  Default   11.000000                  orgs   11.000000   2.0000   2.0000
889  Default   26.000000                  want   26.000000   1.0000   1.0000
..       ...         ...                   ...         ...      ...      ...
288   Topic3    3.527255               goodwin    4.112702   1.1993  -5.8859
397   Topic3    3.527255           kim goodwin    4.112702   1.1993  -5.8859
693   Topic3    3.526336  researcher different    4.112350   1.1991  -5.8862
735   Topic3    3.522396               sharing    4.113478   1.1977  -5.8873
732   Topic3    3.511522                   set    4.113507   1.1946  -5.8904
188   Topic3    3.503869         diverse group    4.114081   1.1923  -5.8925
499   Topic3    4.188803                 mixed    4.932974   1.1893  -5.7140
832   Topic3    6.814088                 tomer    8.213368   1.1660  -5.2274
158   Topic3    7.339972                   day    9.089237   1.1391  -5.1531
736   Topic3    5.935992                sharon    7.394905   1.1331  -5.3654
303   Topic3    9.038417                 group   12.323398   1.0428  -4.9449
824   Topic3    7.718688               thought   12.431304   0.8762  -5.1028
785   Topic3    5.540375               startup    8.224825   0.9577  -5.4344
786   Topic3   10.494819                 steve   22.213930   0.6030  -4.7955
439   Topic3    4.749443                 leisa    6.578592   1.0270  -5.5884
692   Topic3   19.493240            researcher   82.210862  -0.0864 

In [42]:
lda_W22 = lda22.transform(ideal_att_matrix)

In [ ]:
top_doc_column = datadf.primary_industry.dropna()

In [ ]:
word_count_matrix2, count_vect2 = nlp.create_wordcount_matrix(datadf.primary_industry, max_df=0.8, min_df=2, ngram=(1,3))

LDA2 = LatentDirichletAllocation(n_components=5, random_state=42)
LDA2.fit(word_count_matrix2)

In [27]:
lda_H = LDA2.transform(word_count_matrix2)

In [ ]:
nlp.find_top_documents_per_topic(lda_H, top_doc_column, 3)